In [42]:
from pymongo import MongoClient
from bson import ObjectId
import urllib.parse
from sqlalchemy import create_engine, Column, Integer, String, Enum, ForeignKey, Table, MetaData
from sqlalchemy.sql.sqltypes import DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import re

In [43]:
db_config = {
    'postgres': {
    'host': 'localhost',
    'database': 'tenshi',
    'username': 'postgres',
    'password': 'postgres',
    },
    'mongo': {
        'host': 'localhost',
        'port': '27017',
        'database': 'tenshi',
        'username': 'root',
        'password': 'root',
        'auth_source': 'admin'
    }
}

In [44]:
def get_mongo_connection(mongo_config):
    """
    Create a MongoDB connection string from the configuration dictionary.
    """
    username = urllib.parse.quote_plus(mongo_config['username'])
    password = urllib.parse.quote_plus(mongo_config['password'])
    connection_string = (f"mongodb://{username}:{password}@"
                         f"{mongo_config['host']}:{mongo_config['port']}/"
                         f"{mongo_config['database']}?authSource={mongo_config['auth_source']}")
    return MongoClient(connection_string)

def get_postgres_engine(postgres_config):
    username = urllib.parse.quote_plus(postgres_config['username'])
    password = urllib.parse.quote_plus(postgres_config['password'])
    connection_string = (f"postgres+psycopg2://{username}:{password}@"
                         f"{postgres_config['host']}/"
                         f"{postgres_config['database']}")
    return create_engine(connection_string, echo=True)
    
    

# Function to convert camel case to snake case
def camel_to_snake(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()


In [36]:
def generate_sql(document):
    # Replace this with the actual implementation of your SQL generation.
    sql_command = document.get("externalId")
    return document

area = {'_id': ObjectId('62c3c48fc526b5bc881fdeba'), 'collection': 'objectTypes', 'externalId': 'areas', 'displayName': 'Area', 'pluralName': 'Areas', 'description': 'These are manufacturing or other general Areas within a Facility', 'properties': [{'_id': ObjectId('62c3c493df0c3944931d0618'), 'externalId': 'externalId', 'displayName': 'Room Number', 'placeholder': 'Enter the Room Number', 'flags': 27, 'sortOrder': 1, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('62c3c497a8f3407da782f3c7'), 'externalId': 'displayName', 'displayName': 'Room Location', 'placeholder': 'Enter the Room Location', 'flags': 21, 'sortOrder': 2, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('62de1a0a6180a404c014dd15'), 'externalId': 'subAreaName', 'displayName': 'Sub-Area Name', 'placeholder': 'Enter the Sub-Area Name', 'flags': 16, 'sortOrder': 3, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('62de1a1964d38b53446bfa72'), 'externalId': 'areaDept', 'displayName': 'Area Dept', 'placeholder': 'Enter the Area Dept', 'flags': 16, 'sortOrder': 2, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('64cb6f13139da9c5f898a3b9'), 'externalId': 'createdAt', 'displayName': 'Created At', 'placeHolder': 'created at time', 'flags': 1, 'sortOrder': 99990, 'inputType': 'DATE_TIME', 'usageStatus': 1}, {'_id': ObjectId('64cb6fc752d1d6a95a891252'), 'externalId': 'updatedAt', 'displayName': 'Updated At', 'placeHolder': 'updated at time', 'flags': 1, 'sortOrder': 99991, 'inputType': 'DATE_TIME', 'usageStatus': 1}, {'_id': ObjectId('64cb701c52d1d6a95a891253'), 'externalId': 'createdBy', 'displayName': 'Created By', 'placeHolder': 'created by user', 'flags': 1, 'sortOrder': 99992, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('64cb708b52d1d6a95a891254'), 'externalId': 'updatedBy', 'displayName': 'Updated By', 'placeHolder': 'Updated by user', 'flags': 1, 'sortOrder': 99993, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': '64cb70c552d1d6a95a891255', 'externalId': 'usageStatus', 'displayName': 'Usage Status', 'placeHolder': 'active or deprecated', 'flags': 1, 'sortOrder': 99994, 'inputType': 'NUMBER', 'usageStatus': 1}], 'statuses': [], 'usageStatus': 1, 'createdAt': 1656200724, 'updatedAt': 1656200724, 'relations': [], 'version': 1}

equipment = {'_id': ObjectId('6351132d5307a07100c59bae'), 'collection': 'objectTypes', 'createdAt': 1666255452, 'description': 'Equipment', 'displayName': 'Equipment', 'externalId': 'equipments', 'pluralName': 'Equipments', 'properties': [{'_id': ObjectId('63510d0a153d55d208673bf6'), 'externalId': 'externalId', 'displayName': 'Equipment ID', 'placeholder': '', 'flags': 27, 'sortOrder': 1, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('63510d1040681d930bcdf802'), 'externalId': 'displayName', 'displayName': 'Equipment Name', 'placeholder': '', 'flags': 21, 'sortOrder': 2, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('63510d1397f1cc1236151bb1'), 'externalId': 'make', 'displayName': 'Make', 'placeholder': '', 'flags': 0, 'sortOrder': 3, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('63510d195fd914b66926d28e'), 'externalId': 'model', 'displayName': 'Model', 'placeholder': '', 'flags': 0, 'sortOrder': 4, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('6425606f0000000000de4a45'), 'externalId': 'qualificationDoneOn', 'displayName': 'Qualification Done On', 'placeholder': '', 'flags': 0, 'sortOrder': 5, 'inputType': 'DATE', 'usageStatus': 1}, {'_id': ObjectId('642561030000000000de4a46'), 'externalId': 'qualificationDueOn', 'displayName': 'Qualification Due On', 'placeholder': '', 'flags': 0, 'sortOrder': 6, 'inputType': 'DATE', 'usageStatus': 1}, {'_id': ObjectId('64cb6f13139da9c5f898a3b9'), 'externalId': 'createdAt', 'displayName': 'Created At', 'placeHolder': 'created at time', 'flags': 1, 'sortOrder': 99990, 'inputType': 'DATE_TIME', 'usageStatus': 1}, {'_id': ObjectId('64cb6fc752d1d6a95a891252'), 'externalId': 'updatedAt', 'displayName': 'Updated At', 'placeHolder': 'updated at time', 'flags': 1, 'sortOrder': 99991, 'inputType': 'DATE_TIME', 'usageStatus': 1}, {'_id': ObjectId('64cb701c52d1d6a95a891253'), 'externalId': 'createdBy', 'displayName': 'Created By', 'placeHolder': 'created by user', 'flags': 1, 'sortOrder': 99992, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('64cb708b52d1d6a95a891254'), 'externalId': 'updatedBy', 'displayName': 'Updated By', 'placeHolder': 'Updated by user', 'flags': 1, 'sortOrder': 99993, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': '64cb70c552d1d6a95a891255', 'externalId': 'usageStatus', 'displayName': 'Usage Status', 'placeHolder': 'active or deprecated', 'flags': 1, 'sortOrder': 99994, 'inputType': 'NUMBER', 'usageStatus': 1}], 'relations': [{'_id': ObjectId('63510d7a60944491943653ed'), 'externalId': 'areas', 'displayName': 'Location', 'description': '', 'target': {'type': 'OBJECTS', 'urlPath': '/objects/partial?collection=areas', 'cardinality': 'ONE_TO_ONE'}, 'variables': None, 'sortOrder': 1, 'flags': 16, 'usageStatus': 1, 'objectTypeId': '62c3c48fc526b5bc881fdeba'}, {'_id': ObjectId('63510d7f1a0b9ff99addbb40'), 'externalId': 'equipmentTypes', 'displayName': 'Equipment Type', 'description': '', 'target': {'type': 'OBJECTS', 'urlPath': '/objects/partial?collection=equipmentTypes', 'cardinality': 'ONE_TO_ONE'}, 'variables': None, 'sortOrder': 2, 'flags': 16, 'usageStatus': 1, 'objectTypeId': '63510a4baf573b465c4ffcdb'}], 'statuses': [], 'updatedAt': 1666255452, 'usageStatus': 1, 'version': 1}

In [38]:
def process_documents(documents):
    """
    Process each document (e.g., generate SQL and print it).
    """
    for document in documents:
        sql = generate_sql(document)
        print(sql)


In [39]:
client = get_mongo_connection(db_config['mongo'])
db = client[db_config['mongo']['database']]
collection = db['objectTypes']
documents = collection.find({'usageStatus': 1}).sort('externalId', 1)
process_documents(documents)

{'_id': ObjectId('6332dc74a39166f5aa08c272'), 'collection': 'objectTypes', 'externalId': 'accessories', 'displayName': 'Accessories', 'pluralName': 'Accessories', 'description': 'Accessories', 'properties': [{'_id': ObjectId('6332dc74a39166f5aa08c273'), 'externalId': 'externalId', 'displayName': 'Accessory Number', 'placeholder': 'Enter the Accessory Number', 'flags': 27, 'sortOrder': 1, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('6332dc74a39166f5aa08c274'), 'externalId': 'displayName', 'displayName': 'Accessory Name', 'placeholder': 'Enter the Accessory Name', 'flags': 21, 'sortOrder': 2, 'inputType': 'SINGLE_LINE', 'usageStatus': 1}, {'_id': ObjectId('64cb6f13139da9c5f898a3b9'), 'externalId': 'createdAt', 'displayName': 'Created At', 'placeHolder': 'created at time', 'flags': 1, 'sortOrder': 99990, 'inputType': 'DATE_TIME', 'usageStatus': 1}, {'_id': ObjectId('64cb6fc752d1d6a95a891252'), 'externalId': 'updatedAt', 'displayName': 'Updated At', 'placeHolder': 'upda